In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.0 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import numpy as np
from sklearn.metrics import f1_score,classification_report, confusion_matrix,precision_score, recall_score
from tensorflow import keras

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts-ontology-manual-label.csv')
df

,post,prefLabel,compulsion,obsession,expand1
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."
...,...,...,...,...,...
9991,https://itunes.apple.com/gb/podcast/dhamma-tal...,"['interest', 'Thought', 'OCD', 'cognitive proc...",0,0,"morbid thoughts, Mental image, Homicidal thoug..."
9992,Hi all\nThere are lots of helpful topics on th...,['OCD'],0,0,NaN
9993,"aww guys i am sorry , i feel that i am always ...","['OCD', 'sadness', 'weakness', 'anxiety', 'anx...",0,0,"castration anxiety, performance anxiety, koro,..."
9994,My name is Ian I have suffered with OCD for mo...,"['OCD', 'hope', 'fear']",1,0,"terror, panic, panic, moral panic"


In [ ]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
df['post'] = df['post'].fillna('')
df['post'] = df['post'].apply(process_text)
df.head()

,post,prefLabel,compulsion,obsession,expand1
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."


In [ ]:
X, Y = df['post'], df[['compulsion']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7996,), (7996, 1), (2000,), (2000, 1))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
num_words = 3000
vect=Tokenizer(num_words=num_words)
vect.fit_on_texts(X_train)
vocab_size = len(vect.word_index) + 1
print(vocab_size)

22629


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoded_docs_train = vect.texts_to_sequences(X_train)
MAX_LEN = 150
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=MAX_LEN, padding='pre')
padded_docs_train.shape

(7996, 150)

In [ ]:
encoded_docs_test =  vect.texts_to_sequences(X_test)
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=MAX_LEN, padding='pre')

In [ ]:
X_test.shape

(2000,)

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

def get_model():
    model = Sequential()
    # Configuring the parameters
    model.add(Embedding(num_words, output_dim=16, input_length=MAX_LEN))
    model.add(LSTM(16, return_sequences=True))
    # Adding a dropout layer
    model.add(Dropout(0.5))
    model.add(LSTM(8))
    # model.add(Dropout(0.5))
    model.add(Dense(4))
    # model.add(Dropout(0.5))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    return model

In [ ]:
compulsion_model = get_model()
compulsion_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = compulsion_model.fit(padded_docs_train, y_train, epochs=1000, batch_size=16,validation_split=0.2,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=20, min_delta=1e-7),
                              keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=10),
                              keras.callbacks.ModelCheckpoint(filepath='model/lstm_compulsion_model.h5',
                                      monitor='val_loss',
                                      save_best_only=True)])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           48000     
                                                                 
 lstm (LSTM)                 (None, 150, 16)           2112      
                                                                 
 dropout (Dropout)           (None, 150, 16)           0         
                                                                 
 lstm_1 (LSTM)               (None, 8)                 800       
                                                                 
 dense (Dense)               (None, 4)                 36        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 50953 (199.04 KB)
Trainable params: 50953 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 73s 172ms/step - loss: 0.4525 - acc: 0.7919 - val_loss: 0.2676 - val_acc: 0.9137 - lr: 0.0010
Epoch 2/1000
400/400 [==============================] - 58s 146ms/step - loss: 0.2248 - acc: 0.9340 - val_loss: 0.2303 - val_acc: 0.9300 - lr: 0.0010
Epoch 3/1000
400/400 [==============================] - 68s 170ms/step - loss: 0.1898 - acc: 0.9487 - val_loss: 0.2034 - val_acc: 0.9475 - lr: 0.0010
Epoch 4/1000
400/400 [==============================] - 50s 126ms/step - loss: 0.1618 - acc: 0.9556 - val_loss: 0.2020 - val_acc: 0.9425 - lr: 0.0010
Epoch 5/1000
400/400 [==============================] - 65s 162ms/step - loss: 0.1453 - acc: 0.9598 - val_loss: 0.2116 - val_acc: 0.9425 - lr: 0.0010
Epoch 6/1000
400/400 [==============================] - 52s 130ms/step - loss: 0.1317 - acc: 0.9626 - val_loss: 0.2513 - val_acc: 0.9212 - lr: 0.0010
Epoch 7/1000
400/400 [==============================] - 52s 130ms/step - loss: 0.1721 - acc: 0.9439 - val_loss: 0

In [ ]:
compulsion_predictions = compulsion_model.predict([padded_docs_test])

63/63 [==============================] - 2s 22ms/step


In [ ]:
# Assuming 'obsession_predictions' contains probability scores
binary_predictions = np.round(compulsion_predictions)

In [ ]:
print(type(y_test), type(compulsion_predictions))

<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming 'y_test' is a Pandas DataFrame
y_test_array = y_test.to_numpy()

# Assuming 'obsession_predictions' contains the predicted labels or probabilities
# Convert predicted probabilities to binary class labels if needed
binary_predictions = np.round(compulsion_predictions)

# Evaluate accuracy
accuracy = accuracy_score(y_test_array, binary_predictions)
print(f'Accuracy: {accuracy:.4f}')

# Generate a classification report
class_report = classification_report(y_test_array, binary_predictions)
print('Classification Report:\n', class_report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test_array, binary_predictions)
print('Confusion Matrix:\n', conf_matrix)


Accuracy: 0.9210
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.94      0.94      1422
           1       0.86      0.87      0.86       578

    accuracy                           0.92      2000
   macro avg       0.90      0.91      0.90      2000
weighted avg       0.92      0.92      0.92      2000

Confusion Matrix:
 [[1337   85]
 [  73  505]]
